In [ ]:
import boto3

kinesis = boto3.client('kinesis')
iam = boto3.client('iam')

In [ ]:
import datetime
import json
import random
import uuid
import time

class PositionWriter:
    def __init__(self, stream_name, kinesis_client):
        self.stream_name = stream_name
        self.kinesis_client = kinesis_client
        
    def write_position(self, owner, symbol, amount):
        data = {
            "owner" : owner,
            "symbol" : symbol,
            "amount" : amount
        }
        
        print(data)
        
        self.kinesis_client.put_record(
            StreamName=self.stream_name,
            Data=json.dumps(data),
            PartitionKey=owner)
        
class QuoteWriter:
    def __init__(self, stream_name, kinesis_client):
        self.stream_name = stream_name
        self.kinesis_client = kinesis_client
        
    def write_quote(self, symbol, price):
        data = {
            'TICKER': symbol,
            'PRICE': price
        }
        
        print(data)
        
        # not needed - used for compatibility with copy/paste stream schema
        cloud_event = {
            "specversion" : "1.0",
            "type" : "tick",
            "source" : "sample-stream",
            "subject" : "delayed-data",
            "id" : str(uuid.uuid4()),
            "time" : datetime.datetime.now().isoformat(),
            "datacontenttype" : "application/json",
            "data" : data
        }
        
        self.kinesis_client.put_record(
            StreamName=self.stream_name,
            Data=json.dumps(cloud_event),
            PartitionKey=symbol # TODO - determine decent parition key given real quote data distribution
        )
        
        

In [ ]:
# Quote write uses 'AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV' for symbols

In [ ]:
positionWriter = PositionWriter("QuoteStream", kinesis)

In [ ]:
positionWriter.write_position("x001","AMZN", 10)

In [ ]:
qw = QuoteWriter("QuoteStream", kinesis)

In [ ]:
qw.write_quote('AMZN', 10)

In [ ]:
qw.write_quote('AMZN', 100)

Stream reader output should contain...

```
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":100.0}'
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":1000.0}'
```

In [ ]:
# Add another position for x001, and a new position for x002
positionWriter.write_position("x001","MSFT", 10)
positionWriter.write_position("x002","MSFT", 20)

In [ ]:
qw.write_quote('MSFT', 10)

Output adds...

```
b'{"OWNER":"x001","SYMBOL":"MSFT","AMOUNT":10.0,"MKT_VALUE":100.0}'
b'{"OWNER":"x002","SYMBOL":"MSFT","AMOUNT":20.0,"MKT_VALUE":200.0}'
```

In [ ]:
# What happens if we change a position? Is it an update or just an add of another value?
positionWriter.write_position("x001","AMZN", 100)

In [ ]:
qw.write_quote('AMZN', 50)

We observe...

```
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":10.0,"MKT_VALUE":1000.0}'
b'{"OWNER":"x001","SYMBOL":"AMZN","AMOUNT":100.0,"MKT_VALUE":10000.0}'
```

How can we treat position writes as upserts?

Trying to write the latest positions to a stream doesn't work, as anything adding in the time window accumulates as a new position, e.g. this won't do it.

```
CREATE OR REPLACE STREAM "POSITIONS_SQL_STREAM" (OWNER VARCHAR(12), SYMBOL VARCHAR(4), AMOUNT REAL);
CREATE OR REPLACE PUMP "STREAM_PUMP" AS INSERT INTO "POSITIONS_SQL_STREAM"
SELECT STREAM OWNER, SYMBOL, AMOUNT
FROM "SOURCE_SQL_STREAM_001"
WHERE OWNER IS NOT NULL;


CREATE OR REPLACE STREAM "LATEST_POSITIONS_STREAM" (OWNER VARCHAR(12), SYMBOL VARCHAR(4), AMOUNT REAL);
CREATE OR REPLACE PUMP "LATEST_POSITIONS_STREAM_PUMP" AS INSERT INTO "LATEST_POSITIONS_STREAM"
SELECT p1.OWNER, p1.SYMBOL, p1.AMOUNT from POSITIONS_SQL_STREAM AS p1 
    JOIN POSITIONS_SQL_STREAM AS p2
    ON (p1.OWNER = p2.OWNER AND p1.ROWTIME >= p2.ROWTIME); 

CREATE OR REPLACE STREAM "QUOTES_SQL_STREAM" (
    TICKER VARCHAR(4), 
    PRICE     DOUBLE);
CREATE OR REPLACE PUMP "STREAM_PUMP2" AS 
  INSERT INTO "QUOTES_SQL_STREAM" 
    SELECT STREAM 
        TICKER,
        PRICE
    FROM "SOURCE_SQL_STREAM_001"
    WHERE OWNER IS NULL;
     
CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" (OWNER VARCHAR(12), SYMBOL VARCHAR(4), AMOUNT REAL, MKT_VALUE REAL);
CREATE OR REPLACE PUMP "OUTPUT_STREAM_PUMP" AS 
INSERT INTO "DESTINATION_SQL_STREAM"
  SELECT STREAM p.OWNER, p.SYMBOL, p.AMOUNT, q.PRICE * p.AMOUNT
    FROM LATEST_POSITIONS_STREAM OVER (RANGE INTERVAL '10' MINUTE PRECEDING) AS p
      JOIN QUOTES_SQL_STREAM AS q
        ON p.SYMBOL = q.TICKER;
```

### Use Lastest Position

We can alter the SQL to pick up just the last position

```
CREATE OR REPLACE STREAM "POSITIONS_SQL_STREAM" (OWNER VARCHAR(12), SYMBOL VARCHAR(4), AMOUNT REAL);
CREATE OR REPLACE PUMP "STREAM_PUMP" AS INSERT INTO "POSITIONS_SQL_STREAM"
SELECT STREAM OWNER, SYMBOL, AMOUNT
FROM "SOURCE_SQL_STREAM_001"
WHERE OWNER IS NOT NULL;

CREATE OR REPLACE STREAM "QUOTES_SQL_STREAM" (
    TICKER VARCHAR(4), 
    PRICE     DOUBLE);
CREATE OR REPLACE PUMP "STREAM_PUMP2" AS 
  INSERT INTO "QUOTES_SQL_STREAM" 
    SELECT STREAM 
        TICKER,
        PRICE
    FROM "SOURCE_SQL_STREAM_001"
    WHERE OWNER IS NULL;
     
CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" (OWNER VARCHAR(12), SYMBOL VARCHAR(4), AMOUNT REAL, MKT_VALUE REAL);
CREATE OR REPLACE PUMP "OUTPUT_STREAM_PUMP" AS 
INSERT INTO "DESTINATION_SQL_STREAM"
  SELECT STREAM p.OWNER, p.SYMBOL, p.AMOUNT, q.PRICE * p.AMOUNT
    FROM POSITIONS_SQL_STREAM OVER (RANGE INTERVAL '10' MINUTE PRECEDING) AS p
    JOIN POSITIONS_SQL_STREAM AS p2
    ON (p.OWNER = p2.OWNER AND p.ROWTIME >= p2.ROWTIME)
      JOIN QUOTES_SQL_STREAM AS q
        ON p.SYMBOL = q.TICKER;
```

Now, restart the application.

In [ ]:
positionWriter.write_position("x001","AMZN", 10)

In [ ]:
qw.write_quote('AMZN', 10)

In [ ]:
qw.write_quote('AMZN', 100)

Running the three above we only see the output for the first quote... what if we include the range in both parts of the positions join?

```
SELECT STREAM p.OWNER, p.SYMBOL, p.AMOUNT, q.PRICE * p.AMOUNT
    FROM POSITIONS_SQL_STREAM OVER (RANGE INTERVAL '10' MINUTE PRECEDING) AS p
    JOIN POSITIONS_SQL_STREAM OVER (RANGE INTERVAL '10' MINUTE PRECEDING)AS p2
```

Now the second quote is picked up. Next, we alter the position.

In [ ]:
positionWriter.write_position("x001","AMZN", 100)

In [ ]:
qw.write_quote('AMZN', 100)

In [ ]:
positionWriter.write_position("x001","AMZN", 50)

Now the latest position is picked up, but multiple times... 